## Importing all necessary libararies

In [5]:
import os
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sentence_transformers import util

### Constants

In [ ]:
#The file path to the dataset
dataFilePath = r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2"

### Reading the input(Essays) as text files and parsing it into sentences and tokens

In [2]:
sentences = []
for filename in os.listdir(dataFilePath):
   with open(os.path.join(dataFilePath, filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           sentences.append(s)

## Reading the gold standards and vectorizing them
df_ref = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\CSV Files\Ads prompt\01.08.22\new Gold standards.xlsx")
listRef = df_ref['Gold standard']

### building the sentences' vectors using TF-IDF vectorizer

In [6]:
vectorizer = TfidfVectorizer(min_df=1, stop_words='english')
vectorizer.fit_transform(sentences)
vectors = vectorizer.transform(sentences)

refs = vectorizer.transform(listRef)

### Iterating over all the sentneces and assigning the gold standard with the highest cosin similarity

In [ ]:
cosListRefToClusters = []
mm=0
# looping over all the sentences
for emb in vectors:
    listMax = []
    ## looping over all the gold standards to calculate the cos_sim between them and the sentences.
    for embref in refs:
        cos_sim = util.cos_sim(emb.toarray(), embref.toarray()) # cos_sim is the value of the cosine similairty
        listMax.append(cos_sim)
    index_max = np.argmax(listMax) #picking up the index (Gold standard Id)  of the max cosine similairty
    cosListRefToClusters.append(index_max)
    df_test = df_test.append({
        "ID" : mm,
        "text": sentences[mm],
        "AssignedSimilairty": index_max,
        "Similairty" : listMax[index_max]
              },ignore_index=True)
    mm = mm+1


df_test.to_excel("Tf-Idf With Centriods.xlsx");